In [1]:
%pwd

'd:\\Projects\\Gen AI\\medical-assistant\\research'

In [2]:
import os 
os.chdir("../")
%pwd

'd:\\Projects\\Gen AI\\medical-assistant'

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [5]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key)
pc

In [6]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

index_name = "medical-experiment-copy"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(index_name, "created")

index = pc.Index(index_name)

medical-experiment-copy created


In [7]:
info = pc.describe_index(index_name)
print(info)

{'deletion_protection': 'disabled',
 'dimension': 384,
 'host': 'medical-experiment-copy-zs4v7ft.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'medical-experiment-copy',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': None,
 'vector_type': 'dense'}


In [8]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# pick one file for testing
pdf_path = "data/patient_data/Patient_P0004.pdf"
patient_id = os.path.basename(pdf_path).replace(".pdf", "")   

loader = PyPDFLoader(pdf_path)
docs = loader.load()

# split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

chunks[0]


Document(metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Patient: P0004', 'source': 'data/patient_data/Patient_P0004.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='Patient:  Arjun  Kumar  \nPatient  ID:  HK-728-455  \nDOB:  15/08/1975  (Age  48  at  first  visit)  \nGender:  Male  \nOccupation:  Software  Project  Manager  \nKnown  Allergies  (Established  prior  to  Visit  1):  \nSulfa  Drugs  (Trimethoprim/Sulfamethoxazole)  -  Causes  generalized  urticaria  (hives)  and  \npruritus\n \n(itching).\n \nNo  known  food  allergies.  \nPast  Medical  History  (PMH):  \nHypertension  (HTN),  diagnosed  5  years  ago,  well-controlled  on  medication.  \nDyslipidemia,  diagnosed  5  years  ago.  \nMedications  on  file  (Pre-Visit  1):  \nTab.  Telmisartan  40  mg  -  once  daily  \nTab.  Atorvastatin  10  mg  -  once  at  night  \nSocial  History:  \nSmokes  occasionally  (5-10  cigarettes/week,  social  

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()
embedding

C:\Users\anime\AppData\Local\Temp\ipykernel_19544\3442840130.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
from langchain_pinecone import PineconeVectorStore

print(patient_id)

vector_store = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embedding,
    index_name=index_name,
    namespace=patient_id
)

Patient_P0004


In [11]:
print(dir(vector_store))
vector_store_index = vector_store._index  # this is the actual Pinecone index
print(vector_store_index.describe_index_stats())


['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_asimilarity_search_with_relevance_scores', '_async_index', '_cosine_relevance_score_fn', '_embedding', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_index', '_index_host', '_max_inner_product_relevance_score_fn', '_namespace', '_pinecone_api_key', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', '_text_key', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_texts', 'adelete', 'afrom_documents', 'afrom_texts', 'aget_by_ids', 'amax_marginal_relevance_search', 'amax_marginal_relevance_search_by_vector', 'as_retriever', 'asear

In [23]:
retriever = vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5, "namespace": patient_id})

In [24]:
retrieved_docs = retriever.invoke("information about visit 1")
retrieved_docs

[Document(id='08b8b87d-9395-4636-b302-55fdad8e7186', metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 3.0, 'page_label': '4', 'producer': 'Skia/PDF m141 Google Docs Renderer', 'source': 'data/patient_data/Patient_P0004.pdf', 'title': 'Patient: P0004', 'total_pages': 5.0}, page_content='Visit  3:  Routine  Management  &  A  New  Problem  \nDate:  December  14,  2023  \nReason  for  Visit:  "Routine  follow-up  for  HTN  and  GERD.  Now  has  a  painful  rash  on  foot."  \nSubjective:  \n"Patient\n \nhere\n \nfor\n \nscheduled\n \nfollow-up.\n \nHe\n \nreports\n \nexcellent\n \nresponse\n \nto\n \nPantoprazole\n \n-\n \nhis\n \nchest\n \nburning\n \nsymptoms\n \nhave\n \ncompletely\n \nresolved.\n \nHis\n \nhome\n \nBP\n \nlog\n \non\n \nincreased\n \nTelmisartan\n \nshows\n \nexcellent\n \ncontrol,\n \nwith\n \naverages\n \nof\n \n128/82\n \nmmHg.\n \nHe\n \nfeels\n \nless\n \nfatigued.\n \nHowever,\n \nhe\n \nnow\n \nreports\n \na\n \nnew,\n \nunrelated\n \nissue:\n \nfor\n \

In [25]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

chatModel = ChatOpenAI(
    model="deepseek/deepseek-chat",    # ✅ safe working model
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    timeout=30,                        # ⏱ fail fast instead of hanging forever
    max_retries=1,                     # no endless retries
    default_headers={
        "HTTP-Referer": "http://localhost:8501",
        "X-Title": "Medical Chatbot 1"
    },
)

print("Sending request...")
resp = chatModel.invoke("Quick test: say hello if you're working! And tell me are you coming from OpenRouter, if no then from where?")
print("Response:", resp.content)


Sending request...
Response: Hello! I'm working perfectly. 😊  

I'm not coming from OpenRouter—I'm directly hosted by DeepSeek, the creators of my AI model. Let me know how I can assist you! 


In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [27]:
system_prompt = (
    "You are a clinical safety and information retrieval assistant. Your function is to find answers in the provided context, not to generate medical knowledge. "
    "STRICT RULES: "
    "1. SOURCE STRICTLY FROM CONTEXT: You are forbidden from using any internal knowledge, pre-training, or general facts to answer medical questions. Your only sources of information are the 'Patient Context' and the 'Medical Knowledge Base' provided below. "
    "2. HALLUCINATION PROHIBITION: Never invent a drug, dosage, interaction, side effect, or treatment guideline. If the information is not explicitly in the provided context, you must state 'The necessary information is not available in the patient's records or the provided medical knowledge base.' "
    "3. SAFETY CHECK PROTOCOL: When asked about a treatment, your only role is to cross-reference the provided context. Check the patient's allergies and medications against the proposed treatment using ONLY the data provided. You cannot validate if a treatment is correct, only if there is a conflict in the data. "
    "4. NO SPECULATION: Do not speculate, infer, or assume. If a piece of information is missing, state it is missing. "
    "5. CONCISE RESPONSES: Provide direct, concise answers. "
    "OPERATIONAL PROCEDURE: "
    "- For patient summaries: Extract and summarize only the facts present in the Patient Context. "
    "- For questions about history: Answer only with direct quotes or precise summaries from the Patient Context. "
    "- For prescription checks: Execute only this sequence: a) Check proposed treatment against patient's allergy list (from Context). b) Check for any documented drug-drug interactions (from Medical Knowledge Base). c) Report any conflicts found. You cannot comment on efficacy or standard of care, only on data conflicts. "
    "FINAL DISCLAIMER: Always end any advisory response with: 'REMINDER: This is a data retrieval tool, not a clinical decision-maker. Verify all information with primary sources and your professional judgment.' "
    "Use the following retrieved context to inform your response. If the answer is not here, it is unknown: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [28]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [32]:
response = rag_chain.invoke({"input": "any blood in cough"})
print(response["answer"])

Based on the provided context, there is no mention of blood in the cough. The patient reports a "dry, hacking cough" and denies symptoms like fever, chills, shortness of breath, or chest pain. The doctor's assessment also does not note any hemoptysis (coughing up blood).  

**REMINDER: This is a data retrieval tool, not a clinical decision-maker. Verify all information with primary sources and your professional judgment.**


In [30]:
response = rag_chain.invoke({"input": "can i prescribe him Cofsils"})
print(response["answer"])

Based on the provided context, there is no information available about the drug 'Cofsils' in either the patient's records or the medical knowledge base. Therefore, I cannot confirm or deny its appropriateness for this patient.

REMINDER: This is a data retrieval tool, not a clinical decision-maker. Verify all information with primary sources and your professional judgment.
